#Cleaning the Feather Dataset

In [1]:
import pandas as pd

# Load the feather file
df = pd.read_feather("app_data_starter_coded_202502.feather")

# Save as Parquet
df.to_parquet("app_data_starter_coded_202502.parquet", index=False)





In [2]:
df.head(10)

,application_number,examiner_id,au,app_filing_date,app_last_action_date,app_disposal_type,gender,race,examiner_earliest_date
0,08284457,96082.0,1764.0,2000-01-26,30jan2003 00:00:00,ISS,female,white,2000-01-10
1,08413193,87678.0,1764.0,2000-10-11,27sep2010 00:00:00,ISS,None,white,2000-01-04
2,08531853,63213.0,1752.0,2000-05-17,30mar2009 00:00:00,ISS,female,white,2000-01-06
3,08637752,73788.0,1648.0,2001-07-20,07sep2009 00:00:00,ISS,female,white,2000-01-04
4,08682726,77294.0,1762.0,2000-04-10,19apr2001 00:00:00,ABN,male,white,2000-01-03
5,08687412,68606.0,1734.0,2000-04-28,16jul2001 00:00:00,ISS,female,white,2000-01-04
6,08716371,89557.0,1627.0,2004-01-26,15may2017 00:00:00,PEND,female,black,2001-12-19
7,08765941,97543.0,1645.0,2000-06-23,03apr2002 00:00:00,ABN,female,white,2000-02-08
8,08776818,98714.0,1637.0,2000-02-04,27nov2002 00:00:00,ABN,female,white,2000-01-21
9,08809677,65530.0,1723.0,2002-02-20,23mar2009 00:00:00,ISS,female,Asian,2000-01-03


# Initial Application Processing Time Computation

In [3]:
df["app_filing_date"] = pd.to_datetime(df["app_filing_date"], errors="coerce")
df["app_last_action_date"] = pd.to_datetime(df["app_last_action_date"], errors="coerce")
df["processing_time"] = (df["app_last_action_date"] - df["app_filing_date"]).dt.days


<ipython-input-3-3dd8a244b448>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["app_last_action_date"] = pd.to_datetime(df["app_last_action_date"], errors="coerce")


# Date Range Filtering and Cleaning

In [ ]:
import pandas as pd

# Define valid datetime range
min_date = pd.Timestamp("1900-01-01")
max_date = pd.Timestamp("2025-01-01")

# Pre-clean filing date
df["app_filing_date"] = pd.to_datetime(df["app_filing_date"], errors="coerce")
df = df[(df["app_filing_date"] >= min_date) & (df["app_filing_date"] <= max_date)]

# Pre-clean last action date
df["app_last_action_date"] = pd.to_datetime(df["app_last_action_date"], errors="coerce")
df = df[(df["app_last_action_date"] >= min_date) & (df["app_last_action_date"] <= max_date)]

# Calculate processing time
df["processing_time"] = (df["app_last_action_date"] - df["app_filing_date"]).dt.days


In [8]:

# Define a safe date range
min_date = pd.Timestamp("1900-01-01")
max_date = pd.Timestamp("2025-01-01")

# Parse and filter dates safely
df["app_filing_date"] = pd.to_datetime(df["app_filing_date"], errors="coerce")
df["app_last_action_date"] = pd.to_datetime(df["app_last_action_date"], errors="coerce")

# Filter out rows with invalid or out-of-bounds dates
df = df[
    (df["app_filing_date"].between(min_date, max_date)) &
    (df["app_last_action_date"].between(min_date, max_date))
]

# Compute processing time
df["processing_time"] = (df["app_last_action_date"] - df["app_filing_date"]).dt.days

# ===============================
# MISSING VALUES SUMMARY
# ===============================
missing = df.isnull().sum().to_frame(name="Missing Count")
missing["Missing %"] = (missing["Missing Count"] / len(df)) * 100
print("Missing Values Summary:\n", missing)

# ===============================
# SUMMARY STATISTICS
# ===============================
print("\nData Summary:\n", df.describe(include='all'))

# ===============================
# VISUALIZATIONS
# ===============================
import seaborn as sns

sns.set(style="whitegrid")

Missing Values Summary:
                         Missing Count  Missing %
application_number                  0   0.000000
examiner_id                      9000   0.446904
au                                  0   0.000000
app_filing_date                     0   0.000000
app_last_action_date                0   0.000000
app_disposal_type                   0   0.000000
gender                         303070  15.049239
race                                0   0.000000
examiner_earliest_date              0   0.000000
processing_time                     0   0.000000

Data Summary:
        application_number   examiner_id            au  \
count             2013856  2.004856e+06  2.013856e+06   
unique            2013856           NaN           NaN   
top              90012338           NaN           NaN   
freq                    1           NaN           NaN   
mean                  NaN  7.871136e+04  1.927817e+03   
min                   NaN  5.901200e+04  1.600000e+03   
25%                  

In [9]:
# Drop all NaNs immediately
df = df.dropna()

In [11]:
# Filter out rows with invalid or out-of-bounds dates
df = df[
    (df["app_filing_date"].between(min_date, max_date)) &
    (df["app_last_action_date"].between(min_date, max_date))
]


In [10]:
df.to_csv("cleaned_feather_data.csv", index=False)


#Cleaning the Edges_Sample.csv Dataset

In [14]:
import pandas as pd
from datetime import datetime
# Load CSV file
df = pd.read_csv("edges_sample.csv")

# Step 1: Parse the date column with coercion of invalid entries
df['advice_date'] = pd.to_datetime(df['advice_date'], errors='coerce')

# Step 2: Filter rows where the date is in a valid range
min_date = pd.Timestamp("1900-01-01")

# Update max_date to today's date
max_date = pd.Timestamp(datetime.today().date())


df = df[df['advice_date'].between(min_date, max_date)]

# Step 3: Drop all rows with any missing (NaN) values
df.dropna(inplace=True)


# Remove self-linked nodes (where ego == alter)
df = df[df["ego_examiner_id"] != df["alter_examiner_id"]]

# Optional Step 4: Save cleaned version
df.to_csv("cleaned_edges_sample.csv", index=False)

import networkx as nx
df_edges = pd.read_csv("cleaned_edges_sample.csv")

# Step 1: Parse and validate the 'advice_date' column
df_edges["advice_date"] = pd.to_datetime(df_edges["advice_date"], errors="coerce")
min_date = pd.Timestamp("1900-01-01")
max_date = pd.Timestamp(datetime.today().date())
df_edges = df_edges[df_edges["advice_date"].between(min_date, max_date)]
df_edges.dropna(inplace=True)

